In [1]:
# These are all the modules I'll be using later. 
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import pandas as pd
import urllib
import cPickle as pickle
from PIL import Image
import scipy.io as sio
import scipy.io
import h5py
import random
import tensorflow as tf
import tensorboard as tb
from sklearn.cross_validation import train_test_split as ttsplit
import math as math
import pylab as P
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (230540, 32, 32, 1) (230540, 6)
Validation set (5214, 32, 32, 1) (5214, 6)
Test set (13068, 32, 32, 1) (13068, 6)


In [3]:
test = train_labels[:64]
t = test[:,1]

In [4]:
batch_size = 64
patch_size = 5
image_size = 32
label_size = 6
num_channels = 1 # gray scale
num_labels = 11
depth1 = 16
depth2 = 32
num_hidden = 64
keep_prob = 0.98

In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels)) / labels.size

In [13]:
graph = tf.Graph()

with graph.as_default():
    tf_train_dataset  = tf.placeholder(tf.float32, shape=[batch_size,image_size,image_size,num_channels])
    tf_train_labels = tf.placeholder(tf.int32, shape=[batch_size,label_size])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size,patch_size,num_channels,depth1],stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size,patch_size,depth1,depth2],stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0,shape=[depth2]))
    #convolution layer that connect after convolution to fully connected output layer
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    #output
    op1_weights = tf.Variable(tf.truncated_normal([num_hidden,num_labels],stddev=0.1))
    op1_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    op2_weights = tf.Variable(tf.truncated_normal([num_hidden,num_labels],stddev=0.1))
    op2_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    op3_weights = tf.Variable(tf.truncated_normal([num_hidden,num_labels],stddev=0.1))
    op3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    op4_weights = tf.Variable(tf.truncated_normal([num_hidden,num_labels],stddev=0.1))
    op4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    op5_weights = tf.Variable(tf.truncated_normal([num_hidden,num_labels],stddev=0.1))
    op5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    #construct model
    def model(data, is_training=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        nomalization = tf.nn.local_response_normalization(hidden)
        pool = tf.nn.max_pool(nomalization,[1,2,2,1],[1,2,2,1],padding = 'SAME')
        
        conv = tf.nn.conv2d(pool, layer2_weights, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        nomalization = tf.nn.local_response_normalization(hidden)
        pool = tf.nn.max_pool(nomalization,[1,2,2,1],[1,2,2,1],padding = 'SAME')
        #final layer before connecting to the output layer
        
        hidden_dropout = tf.nn.dropout(pool, keep_prob)
        shape = hidden_dropout.get_shape().as_list()
        reshaped = tf.reshape(hidden_dropout,[shape[0],shape[1]*shape[2]*shape[3]])
        
        reshaped = tf.nn.relu(tf.matmul(reshaped, layer3_weights) + layer3_biases)
        
        logits1 = tf.matmul(reshaped,op1_weights) + op1_biases
        logits2 = tf.matmul(reshaped,op2_weights) + op2_biases
        logits3 = tf.matmul(reshaped,op3_weights) + op3_biases
        logits4 = tf.matmul(reshaped,op4_weights) + op4_biases
        logits5 = tf.matmul(reshaped,op5_weights) + op5_biases

        return (logits1,logits2,logits3,logits4,logits5)
    logits1,logits2,logits3,logits4,logits5 = model(tf_train_dataset,True)
    loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])
     +  tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])
     +  tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])
     +  tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4])
     +  tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, tf_train_labels[:,5])
    )
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.15, global_step, 1000, 0.93)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    train_prediction = tf.pack([tf.nn.softmax(logits1), tf.nn.softmax(logits2), tf.nn.softmax(logits3),
                              tf.nn.softmax(logits4),tf.nn.softmax(logits5)])
    valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_dataset)[0]), tf.nn.softmax(model(tf_valid_dataset)[1]),
                              tf.nn.softmax(model(tf_valid_dataset)[2]), tf.nn.softmax(model(tf_valid_dataset)[3]),
                              tf.nn.softmax(model(tf_valid_dataset)[4])])
    test_prediction = tf.pack([tf.nn.softmax(model(tf_test_dataset)[0]), tf.nn.softmax(model(tf_test_dataset)[1]),
                             tf.nn.softmax(model(tf_test_dataset)[2]), tf.nn.softmax(model(tf_test_dataset)[3]),
                             tf.nn.softmax(model(tf_test_dataset)[4])])
    saver = tf.train.Saver()
        
    

In [78]:
bathch_la = train_labels[:64,:]

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as sess:
  tf.initialize_all_variables().run()



  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0): 
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:,1:6]))
  final_predictions = test_prediction.eval()
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))

Initialized
Minibatch loss at step 0: 14.762002
Minibatch accuracy: 10.0%
Validation accuracy: 60.4%
Minibatch loss at step 500: 6.233685
Minibatch accuracy: 56.9%
Validation accuracy: 60.2%
Minibatch loss at step 1000: 4.992568
Minibatch accuracy: 68.1%
Validation accuracy: 71.7%
